In [1]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

!pip install -U adapter-transformers
!pip install datasets

     |████████████████████████████████| 2.9 MB 8.2 MB/s 
     |████████████████████████████████| 596 kB 53.4 MB/s 
     |████████████████████████████████| 895 kB 57.4 MB/s 
     |████████████████████████████████| 3.3 MB 54.5 MB/s 
     |████████████████████████████████| 61 kB 608 kB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 298 kB 7.1 MB/s 
     |████████████████████████████████| 243 kB 65.6 MB/s 
     |████████████████████████████████| 1.1 MB 61.6 MB/s 
     |████████████████████████████████| 132 kB 94.0 MB/s 
     |████████████████████████████████| 192 kB 72.7 MB/s 
     |████████████████████████████████| 271 kB 71.4 MB/s 
     |████████████████████████████████| 160 kB 71.7 MB/s 


In [2]:
from datasets import load_dataset, DatasetDict

dataset = load_dataset("yxchar/sciie-tlm")
dataset.num_rows

Using custom data configuration yxchar___sciie-tlm-a32f1f2c4e9b5c0d


  0%|          | 0/3 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/663k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/199k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

  0%|          | 0/3 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/yxchar___sciie-tlm-a32f1f2c4e9b5c0d/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

{'test': 974, 'train': 3219, 'validation': 455}

In [3]:
# max([len(x['text']) for x in dataset['train']])

In [ ]:
# dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'text', 'id', 'label'],
        num_rows: 3219
    })
    test: Dataset({
        features: ['Unnamed: 0', 'text', 'id', 'label'],
        num_rows: 974
    })
    validation: Dataset({
        features: ['Unnamed: 0', 'text', 'id', 'label'],
        num_rows: 455
    })
})

In [4]:
from transformers import RobertaTokenizer

tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

def encode_batch(batch):
  """Encodes a batch of input data using the model tokenizer."""
  return tokenizer(batch["text"], max_length=512, truncation=True, padding="max_length")

# Encode the input data
dataset = dataset.map(encode_batch, batched=True)
# The transformers model expects the target class column to be named "labels"
dataset.rename_column_("label", "labels")
# Transform to pytorch tensors and only output the required columns
dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: FutureWarning: rename_column_ is deprecated and will be removed in the next major version of datasets. Use DatasetDict.rename_column instead.
  if sys.path[0] == '':


In [ ]:
# dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'attention_mask', 'id', 'input_ids', 'labels', 'text'],
        num_rows: 3219
    })
    test: Dataset({
        features: ['Unnamed: 0', 'attention_mask', 'id', 'input_ids', 'labels', 'text'],
        num_rows: 974
    })
    validation: Dataset({
        features: ['Unnamed: 0', 'attention_mask', 'id', 'input_ids', 'labels', 'text'],
        num_rows: 455
    })
})

In [5]:
import numpy as np

In [ ]:
# np.unique(dataset['train']['labels'])

array([0, 1, 2, 3, 4, 5, 6])

In [6]:
from transformers import RobertaConfig, RobertaModelWithHeads

config = RobertaConfig.from_pretrained(
    "roberta-base",
    num_labels=7,
)
model = RobertaModelWithHeads.from_pretrained(
    "roberta-base",
    config=config,
)

Downloading:   0%|          | 0.00/478M [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModelWithHeads: ['lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModelWithHeads from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModelWithHeads from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModelWithHeads were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.embeddings.position_ids']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and infere

In [ ]:
# model.load_adapter("/content/final_adapter")

'sciie'

In [7]:
# Add a new adapter
model.add_adapter("sciie")
# Add a matching classification head
model.add_classification_head(
    "sciie",
    num_labels=7,
    # id2label={ 0: "👎", 1: "👍"}
  )

# Activate the adapter
model.train_adapter("sciie")

In [ ]:
# adapter_name = model.load_adapter("/content/final_adapter")

# model.set_active_adapters(adapter_name)

Overwriting existing adapter 'sciie'.
Overwriting existing head 'sciie'


In [ ]:
# import torch
# device='cuda'

In [ ]:
# test_dataset = dataset['test']
# bsz = 16
# i = 0
# batches = []
# while i<test_dataset.num_rows:
#   batches.append(test_dataset[i:i+bsz])
#   i+=bsz

  

In [ ]:
# # Put model in evaluation mode
# model.to(device)
# model.eval()


# # Tracking variables for storing ground truth and predictions 
# predictions , true_labels = [], []

# # Prediction Loop
# for batch in batches:

 
 
#   # Unpack the inputs from our dataloader and move to GPU/accelerator 
 
#   input_ids = batch['input_ids'].to(device)
#   attention_mask = batch['attention_mask'].to(device)
#   labels = batch['labels'].to(device)

  
#   # Telling the model not to compute or store gradients, saving memory and 
#   # speeding up prediction
#   with torch.no_grad():
#       # Forward pass, calculate logit predictions
#       outputs = model(input_ids, attention_mask=attention_mask, 
#                          labels=labels)

#   logits = outputs[1]

#   # Move logits and labels to CPU
#   logits = logits.detach().cpu().numpy()
#   label_ids = labels.to('cpu').numpy()
  
#   # Store predictions and true labels
#   predictions.append(logits)
#   true_labels.append(label_ids)

In [ ]:
# outputs

SequenceClassifierOutput([('loss', tensor(0.0090, device='cuda:0')),
                          ('logits',
                           tensor([[ 0.0283,  0.7030, -3.7663,  7.7852,  0.5362, -2.8256, -3.8829],
                                   [ 0.0699, -0.1707, -5.3138,  8.8135, -0.7966, -1.1407, -2.4159],
                                   [ 0.0486,  0.5042, -4.9819,  8.2899, -0.7819, -1.8018, -2.2783],
                                   [ 8.0929, -2.2417, -2.3154,  1.1259, -2.0502,  1.7409, -4.1980],
                                   [-1.6333,  3.1698, -4.4894,  7.0141, -0.3583, -3.3101, -1.1039],
                                   [-1.4175, -0.4475, -4.6082,  7.8073, -1.1141,  0.0694, -1.1756],
                                   [-2.5500, -0.9112, -2.5983,  8.7986, -0.7085, -2.2642, -1.0847],
                                   [-3.1045, -0.2582, -3.7888,  9.1103, -1.4748, -1.3201, -0.3917],
                                   [ 6.6718, -3.2633, -1.4122,  0.2773, -1.3010,  3.6007, -3.9

In [ ]:
# from datasets import list_metrics
# metrics_list = list_metrics()
# # len(metrics_list)

# print(', '.join(metric for metric in metrics_list))

accuracy, bertscore, bleu, bleurt, cer, chrf, code_eval, comet, competition_math, coval, cuad, f1, gleu, glue, google_bleu, indic_glue, matthews_correlation, meteor, pearsonr, precision, recall, rouge, sacrebleu, sari, seqeval, spearmanr, squad, squad_v2, super_glue, ter, wer, wiki_split, xnli


In [8]:
import numpy as np
from transformers import TrainingArguments, AdapterTrainer, EvalPrediction

training_args = TrainingArguments(
    learning_rate=1e-4,
    num_train_epochs=12,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    logging_steps=200,
    output_dir="./training_output",
    overwrite_output_dir=True,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=False,
)

def compute_accuracy(p: EvalPrediction):
  preds = np.argmax(p.predictions, axis=1)
  return {"acc": (preds == p.label_ids).mean()}

trainer = AdapterTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    compute_metrics=compute_accuracy,
)

In [9]:
# dir(AdapterTrainer)

In [10]:
trainer.train()

***** Running training *****
  Num examples = 3219
  Num Epochs = 12
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 2424


Step,Training Loss
200,1.510800
400,1.044300
600,0.726600
800,0.589600
1000,0.482700
1200,0.418500
1400,0.393800
1600,0.355300
1800,0.307600
2000,0.302500


Saving model checkpoint to ./training_output/checkpoint-500
Configuration saved in ./training_output/checkpoint-500/sciie/adapter_config.json
Module weights saved in ./training_output/checkpoint-500/sciie/pytorch_adapter.bin
Configuration saved in ./training_output/checkpoint-500/sciie/head_config.json
Module weights saved in ./training_output/checkpoint-500/sciie/pytorch_model_head.bin
Configuration saved in ./training_output/checkpoint-500/sciie/head_config.json
Module weights saved in ./training_output/checkpoint-500/sciie/pytorch_model_head.bin
Configuration saved in ./training_output/checkpoint-500/sciie/head_config.json
Module weights saved in ./training_output/checkpoint-500/sciie/pytorch_model_head.bin
Saving model checkpoint to ./training_output/checkpoint-1000
Configuration saved in ./training_output/checkpoint-1000/sciie/adapter_config.json
Module weights saved in ./training_output/checkpoint-1000/sciie/pytorch_adapter.bin
Configuration saved in ./training_output/checkpoint-

TrainOutput(global_step=2424, training_loss=0.5529175628923347, metrics={'train_runtime': 1541.6739, 'train_samples_per_second': 25.056, 'train_steps_per_second': 1.572, 'total_flos': 1.034014219978752e+16, 'train_loss': 0.5529175628923347, 'epoch': 12.0})

In [11]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 455
  Batch size = 16


{'epoch': 12.0,
 'eval_acc': 0.8615384615384616,
 'eval_loss': 0.3747590482234955,
 'eval_runtime': 8.9162,
 'eval_samples_per_second': 51.031,
 'eval_steps_per_second': 3.253}

In [12]:
from transformers import TextClassificationPipeline

classifier = TextClassificationPipeline(model=model, tokenizer=tokenizer, device=training_args.device.index)

classifier("This is awesome!")

[{'label': 'LABEL_3', 'score': 0.46924084424972534}]

In [13]:
model.save_adapter("./final_adapter", "sciie")

!ls -lh final_adapter

Configuration saved in ./final_adapter/adapter_config.json
Module weights saved in ./final_adapter/pytorch_adapter.bin
Configuration saved in ./final_adapter/head_config.json
Module weights saved in ./final_adapter/pytorch_model_head.bin


total 5.8M
-rw-r--r-- 1 root root  571 Dec 15 20:20 adapter_config.json
-rw-r--r-- 1 root root  477 Dec 15 20:20 head_config.json
-rw-r--r-- 1 root root 3.5M Dec 15 20:20 pytorch_adapter.bin
-rw-r--r-- 1 root root 2.3M Dec 15 20:20 pytorch_model_head.bin


In [14]:
from google.colab import files
!zip -r /content/file.zip /content
files.download("/content/file.zip")

  adding: content/ (stored 0%)
  adding: content/.config/ (stored 0%)
  adding: content/.config/gce (stored 0%)
  adding: content/.config/active_config (stored 0%)
  adding: content/.config/configurations/ (stored 0%)
  adding: content/.config/configurations/config_default (deflated 15%)
  adding: content/.config/config_sentinel (stored 0%)
  adding: content/.config/.last_survey_prompt.yaml (stored 0%)
  adding: content/.config/.last_update_check.json (deflated 24%)
  adding: content/.config/logs/ (stored 0%)
  adding: content/.config/logs/2021.12.03/ (stored 0%)
  adding: content/.config/logs/2021.12.03/14.32.30.027140.log (deflated 91%)
  adding: content/.config/logs/2021.12.03/14.33.37.701606.log (deflated 53%)
  adding: content/.config/logs/2021.12.03/14.33.36.903459.log (deflated 54%)
  adding: content/.config/logs/2021.12.03/14.33.16.964195.log (deflated 54%)
  adding: content/.config/logs/2021.12.03/14.33.09.955489.log (deflated 86%)
  adding: content/.config/logs/2021.12.03/14.

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [15]:
pred, label, metrics = trainer.predict(dataset["test"])
pred_label = np.argmax(pred, axis = 1)

from sklearn.metrics import classification_report 
print(classification_report(label, pred_label))

***** Running Prediction *****
  Num examples = 974
  Batch size = 16


              precision    recall  f1-score   support

           0       0.91      0.96      0.93       123
           1       0.69      0.61      0.65        59
           2       0.86      0.90      0.88        67
           3       0.93      0.91      0.92       533
           4       0.60      0.52      0.56        63
           5       0.76      0.84      0.80        38
           6       0.75      0.88      0.81        91

    accuracy                           0.86       974
   macro avg       0.79      0.80      0.79       974
weighted avg       0.86      0.86      0.86       974

